In [1]:
import pandas as pd

## Overview of Met Police dataset (on street crime)

In [2]:
met_df = pd.read_csv('../data/metpolice/2018-07-metropolitan-street.csv').dropna(subset=['Longitude', 'Latitude'])

print(met_df.info())

met_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94577 entries, 0 to 94576
Data columns (total 12 columns):
Crime ID                 73388 non-null object
Month                    94577 non-null object
Reported by              94577 non-null object
Falls within             94577 non-null object
Longitude                94577 non-null float64
Latitude                 94577 non-null float64
Location                 94577 non-null object
LSOA code                94577 non-null object
LSOA name                94577 non-null object
Crime type               94577 non-null object
Last outcome category    73388 non-null object
Context                  0 non-null float64
dtypes: float64(3), object(9)
memory usage: 9.4+ MB
None


,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,e9fe81ec7a6f5d2a80445f04be3d7e92831dbf3090744e...,2018-07,Metropolitan Police Service,Metropolitan Police Service,0.774271,51.148147,On or near Bethersden Road,E01024031,Ashford 012B,Other theft,Under investigation,NaN
1,076b796ba1e1ba3f69c9144e2aa7a7bc85b61d51bf7a59...,2018-07,Metropolitan Police Service,Metropolitan Police Service,-1.007293,51.893136,On or near Prison,E01017674,Aylesbury Vale 010D,Other crime,Awaiting court outcome,NaN
2,163e996d58995cf87d14f15711fbd87052681919f02029...,2018-07,Metropolitan Police Service,Metropolitan Police Service,0.744706,52.038219,On or near Hillside Road,E01029918,Babergh 007F,Violence and sexual offences,Under investigation,NaN
3,NaN,2018-07,Metropolitan Police Service,Metropolitan Police Service,0.137065,51.583672,On or near Police Station,E01000027,Barking and Dagenham 001A,Anti-social behaviour,NaN,NaN
4,NaN,2018-07,Metropolitan Police Service,Metropolitan Police Service,0.148434,51.595164,On or near Park/Open Space,E01000027,Barking and Dagenham 001A,Anti-social behaviour,NaN,NaN


## Get crime frequency per coordiate

In [3]:
met_df['Coordinate'] = met_df.apply(lambda x: (x['Longitude'], x['Latitude']), axis=1)

In [4]:
crime_freq = met_df.groupby('Coordinate').size()
crime_freq.count()

33287

In [5]:
crime_freq.head()

Coordinate
(-5.482662, 50.211896)              2
(-3.3954779999999998, 51.951301)    1
(-3.191991, 51.46346)               1
(-3.016167, 53.3255)                1
(-2.9570790000000002, 53.473227)    1
dtype: int64

In [6]:
print(crime_freq.min())
print(crime_freq.max())

1
239


## Convert data to GeoJSON in order to use Mapbox API

In [7]:
import json

with open('./template.geojson', 'r') as template:
    template = json.load(template)
    
    for index, value in crime_freq.iteritems():
        data = {
            'type': 'Feature',
            'properties': {
                'crimes': value
            },
            'geometry': {
                'type': 'Point',
                'coordinates': [index[0], index[1]]
            }
        }
        
        template['features'].append(data)

with open('../london_july_2018.geojson', 'w') as output:
    json.dump(template, output)